# 数据操作

首先介绍张量（tenser），其实就是 $n$ 维的 array，但是在深度学习框架中叫张量类。

深度学习框架又比 Numpy 的 ndarray 多一些重要功能：

- 首先，GPU很好地支持加速计算，而NumPy仅支持CPU计算； 
- 其次，张量类支持自动微分。这些功能使得张量类更适合深度学习。如果没有特殊说明，本书中所说的张量均指的是张量类的实例。

## 张量入门

和 Numpy 差不多

In [2]:
import torch

x = torch.arange(12)
x

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [8]:
x.shape # size of the tenser

torch.Size([12])

In [9]:
x.numel() # number of the element in a tensor

12

In [11]:
x.reshape(-1, 4) # same as numpy

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])

In [12]:
torch.zeros(2, 3, 4) # same as numpy

tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])

In [13]:
torch.randn(3, 4) # random: standard Guassion distribution

tensor([[ 0.8137, -1.2267, -0.7105,  0.3359],
        [-0.3480,  0.3728,  0.7156,  0.5010],
        [-0.7090, -0.2571,  0.2815,  0.3627]])

In [15]:
torch.tensor([[1, 2, 3, 4], [5, 6, 7, 8]])

tensor([[1, 2, 3, 4],
        [5, 6, 7, 8]])

注意下这个：连结（concatenate）两个张量：

 - `dim=0` 就是沿着上下方向
 - `dim=1` 就是沿着左右方向

In [24]:
X = torch.arange(12, dtype=torch.float32).reshape((3, 4))
Y = torch.tensor([[2.0, 1, 4, 3], [1, 2, 3, 4], [4, 3, 2, 1]])

torch.cat((X, Y), dim=0), torch.cat((X, Y), dim=1)

# From the result we can see

(tensor([[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [ 2.,  1.,  4.,  3.],
         [ 1.,  2.,  3.,  4.],
         [ 4.,  3.,  2.,  1.]]),
 tensor([[ 0.,  1.,  2.,  3.,  2.,  1.,  4.,  3.],
         [ 4.,  5.,  6.,  7.,  1.,  2.,  3.,  4.],
         [ 8.,  9., 10., 11.,  4.,  3.,  2.,  1.]]))

In [26]:
X.sum() # Note that the result is still a tenser type

tensor(66.)

### 广播机制

Numpy 里没认真看，在这看看。

对于尺寸相同的张量，我们都可以进行逐元素操作。但是对于尺寸不同的呢？广播机制在某些情况下也可以。这种机制的工作方式如下：首先，通过适当复制元素来扩展一个或两个数组，以便在转换之后，两个张量具有相同的形状。其次，对生成的数组执行按元素操作。

在大多数情况下，我们将沿着数组中**长度为1的轴**进行广播，如下例子：

In [27]:
a = torch.arange(3).reshape((-1, 1))
b = torch.arange(2).reshape((1, -1))

a, b

(tensor([[0],
         [1],
         [2]]),
 tensor([[0, 1]]))

In [28]:
a + b

tensor([[0, 1],
        [1, 2],
        [2, 3]])

沿着长度为 1 的轴进行广播。也就是就将单维复制为多维：

`a` 被复制为 `[[0, 0], [1, 1], [2, 2]]`，也就是由一列复制为了两列

`b` 被复制为 `[[0, 1], [0, 1], [0, 1]]`，也就是一行复制为了三行

但是这个操作并没有改变原来的张量，而是为了计算临时改变的

## 节省内存

运行一些操作可能会导致为新结果分配内存。例如，如果我们用 `Y = X + Y`，我们将取消引用 `Y` 指向的张量，而是指向新分配的内存处的张量。

可以实验演示，确实是这样：

In [29]:
before = id(Y)
Y = Y + X
id(Y) == before

False

在深度学习中，我们认为这是不好的，因为数据常常会更新，而且数据量很大，这么做可能会导致一些错误。

所以我们的解决方法就是执行原地操作，方法也很简单，使用切片表示法或者 `+=` 此类运算符即可：

In [30]:
# Method 1
before = id(X)
X[:] = X + Y
id(X) == before

True

In [31]:
# Method 2
before = id(X)
X += Y
id(X) == before

True

## 转化为其他对象

事实上 Numpy 数组与 Pytorch 张量共享底层，所以二者基本混着用没什么问题，而且也有内建函数：

In [32]:
A = X.numpy()
B = torch.tensor(A)
type(A), type(B)

(numpy.ndarray, torch.Tensor)

使用其中的常数也很容易：

In [37]:
aa = torch.tensor([3.7])
aa, aa.item()

(tensor([3.7000]), 3.700000047683716)

## 练习